In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder  # <-- add this
from tensorflow.keras.utils import to_categorical

2025-05-07 02:05:35.995669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746583536.019329     327 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746583536.026670     327 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Step 1: Load the Pre-trained Vision Transformer Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)
model.eval()  # Set the model to evaluation mode

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (d

In [3]:
!pip3 install torchinfo
from transformers import ViTModel
from torchinfo import summary

# Load the pre-trained Vision Transformer (ViT) model
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# Get the model summary
summary(model, input_size=(1, 3, 224, 224))  # Batch size of 1, RGB channels, 224x224 image


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                             Output Shape              Param #
ViTModel                                           [1, 768]                  --
├─ViTEmbeddings: 1-1                               [1, 197, 768]             152,064
│    └─ViTPatchEmbeddings: 2-1                     [1, 196, 768]             --
│    │    └─Conv2d: 3-1                            [1, 768, 14, 14]          590,592
│    └─Dropout: 2-2                                [1, 197, 768]             --
├─ViTEncoder: 1-2                                  [1, 197, 768]             --
│    └─ModuleList: 2-3                             --                        --
│    │    └─ViTLayer: 3-2                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-3                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-4                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-5                          [1, 197, 768]             7,087,8

In [ ]:
import os
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
def load_images_and_labels(folder_path, target_size=(224, 224), num_workers=8):
    image_paths = []
    labels = []
    # Scan all image paths and labels
    for label_name in os.listdir(folder_path):
        label_folder = os.path.join(folder_path, label_name)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(label_folder, filename)
                    image_paths.append(image_path)
                    labels.append(label_name)
    # Define a function for parallel loading
    def load_and_preprocess(path):
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, target_size)
            img = img.astype(np.float32) / 255.0  # Normalize inside
        return img
    # Load images in parallel
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        images = list(executor.map(load_and_preprocess, image_paths))

    # Remove any images that failed to load (just in case)
    images = np.array([img for img in images if img is not None], dtype=np.float32)
    labels = np.array(labels)
    return images, labels
# --- Now load your datasets ---
train_folder = "/kaggle/input/lc25000/lung_colon_image_set/Train and Validation Set"
test_folder = "/kaggle/input/lc25000/lung_colon_image_set/Test Set"
x_train, y_train = load_images_and_labels(train_folder)
x_test, y_test = load_images_and_labels(test_folder)

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode string labels to integers [0, 1, 2, 3, 4]
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# One-hot encode labels
#num_classes = len(label_encoder.classes_)
#y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

# Check shapes
print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"x_test: {x_test.shape}, y_test: {y_test.shape}")

In [ ]:
import os
import numpy as np
import torch
from PIL import Image, UnidentifiedImageError
from concurrent.futures import ThreadPoolExecutor

def process_image(img_path, feature_extractor, model, device):
    try:
        # Load and preprocess image
        image = Image.open(img_path).convert("RGB")

        # Preprocess with feature extractor
        inputs = feature_extractor(image, return_tensors="pt").to(device)

        # Extract features
        with torch.no_grad():
            outputs = model(**inputs)

        # Take CLS token feature
        cls_token_feature = outputs.last_hidden_state[:, 0, :]  # CLS token
        return cls_token_feature.cpu().numpy().flatten()

    except UnidentifiedImageError:
        print(f"Warning: Unable to identify image file: {img_path}. Skipping.")
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
    return None
def extract_features(image_paths, feature_extractor, model, device, num_workers=4):
    features = []

    # Define a helper function that wraps process_image
    def process(img_path):
        return process_image(img_path, feature_extractor, model, device)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(process, image_paths))

    # Remove failed images
    features = np.array([res for res in results if res is not None])
    return features

In [ ]:
train_folder = "/kaggle/input/lc25000/lung_colon_image_set/Train and Validation Set"
train_paths = []  # list all your image paths
# Example of collecting all image paths
for root, _, files in os.walk(train_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            train_paths.append(os.path.join(root, file))
# Extract features
features_train = extract_features(train_paths, feature_extractor, model, device, num_workers=8)
print("Extracted feature shape:", features_train.shape)


In [ ]:
import time
start=time.time()
test_folder = "/kaggle/input/lc25000/lung_colon_image_set/Test Set"
test_paths = []  # list all your image paths

# Example of collecting all image paths
for root, _, files in os.walk(test_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            test_paths.append(os.path.join(root, file))
# Extract features
features_test = extract_features(test_paths, feature_extractor, model, device, num_workers=8)
print("Extracted feature shape:", features_test.shape)
end=time.time()
print('Thoi gian thu thi',end-start)
# Lưu vào file Excel
#df = pd.DataFrame(Train_features)  # Chuyển thành DataFrame
#df.to_excel("Train_features.xlsx", index=False)  # Lưu vào tệp Excel

XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import time

st = time.time()
# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # Fit & transform y_train
y_test_encoded = label_encoder.transform(y_test)        # Only transform y_test

# Step 5: Train an XGBoost Classifier
classifier = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
classifier.fit(features_train, y_train_encoded)  # Training with encoded labels  

# Step 6: Make Predictions
y_pred_encoded = classifier.predict(features_test)

# Step 7: Decode predictions back to original labels
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Step 8: Evaluate the Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

nd = time.time()
print("Thoi gian thu thi:", nd - st)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Step 7: Evaluate the Model and Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred)  # Define conf_matrix here
class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
classA =[0,1,2,3,4]
# Map integer labels to class names
categories = [class_names[label] for label in classA]
# Figure 1: Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_XGBoost.png')
plt.show

**Enhanced bayesian method**

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KernelDensity
import numpy as np
import time
st=time.time()
class KDE_NaiveBayes:
    def __init__(self, bandwidth=0.09, default_class=None):
        self.bandwidth = bandwidth
        self.kde_models = {}
        self.classes = None
        self.priors = None
        self.default_class = default_class  # Fallback class if no valid prediction

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = {}
        n_samples = len(y)

        # Calculate prior probabilities
        for c in self.classes:
            self.priors[c] = np.sum(y == c) / n_samples
        
        # Fit KDE for each feature and each class, skip if no samples
        for c in self.classes:
            X_c = X[y == c]
            if X_c.shape[0] == 0:
                print(f"Warning: No samples found for class {c}. Skipping KDE fitting.")
                self.kde_models[c] = None
                continue
            self.kde_models[c] = []
            for feature_idx in range(X.shape[1]):
                kde = KernelDensity(bandwidth=self.bandwidth, kernel='gaussian')
                kde.fit(X_c[:, feature_idx].reshape(-1, 1))
                self.kde_models[c].append(kde)

    def predict(self, X):
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, len(self.classes)))

        for idx, c in enumerate(self.classes):
            log_prior = np.log(self.priors[c]) if self.priors[c] > 0 else -np.inf
            if self.kde_models[c] is None:
                log_probs[:, idx] = -np.inf
                continue
            log_likelihood = np.zeros(n_samples)
            for feature_idx, kde in enumerate(self.kde_models[c]):
                feature_log_prob = kde.score_samples(X[:, feature_idx].reshape(-1, 1))
                log_likelihood += feature_log_prob
            log_probs[:, idx] = log_prior+log_likelihood
        
        # Check if all probabilities are -inf
        if np.all(log_probs == -np.inf, axis=1).any():
            if self.default_class is not None:
                print(f"Warning: Some samples have no valid class probabilities. Using default class: {self.default_class}")
                y_pred = np.array([self.default_class if np.all(log_probs[i] == -np.inf) else self.classes[np.argmax(log_probs[i])] for i in range(n_samples)])
            else:
                raise ValueError("No valid predictions possible for some samples; all classes have zero probability. Set a default_class to handle this.")
        else:
            y_pred = self.classes[np.argmax(log_probs, axis=1)]
        
        return y_pred

# Example usage
classifier = KDE_NaiveBayes(bandwidth=0.09, default_class='Normal')  # Set a default class
classifier.fit(features_train, y_train)
y_pred = classifier.predict(features_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
nd=time.time()
print("Thoi gian",nd-st)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Step 7: Evaluate the Model and Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)  # Define conf_matrix here
class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
classA =[0,1,2,3,4]
# Map integer labels to class names
categories = [class_names[label] for label in classA]
# Figure 1: Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_Naivebayes.png')
plt.show
# plt.close()
# print("Confusion matrix figure saved as 'confusion_matrix.png'.")

In [ ]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Bắt đầu đo thời gian
st = time.time()

# Step 5: Train a Logistic Regression Classifier  
classifier = LogisticRegression(max_iter=1000)  # Tạo đối tượng LogisticRegression, điều chỉnh max_iter nếu cần thiết  
classifier.fit(features_train, y_train)  # Huấn luyện mô hình

# Step 6: Make Predictions  
y_pred = classifier.predict(features_test)  # Dự đoán với dữ liệu kiểm tra

# Step 7: Evaluate the Model  
print("Accuracy:", accuracy_score(y_test, y_pred))  # Đánh giá độ chính xác
print(classification_report(y_test, y_pred))  # In báo cáo phân loại với các chỉ số chính

# Kết thúc đo thời gian
nd = time.time()
print("Thời gian thực thi:", nd - st)  # In ra thời gian thực thi

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Step 7: Evaluate the Model and Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)  # Define conf_matrix here
class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
classA =[0,1,2,3,4]
# Map integer labels to class names
categories = [class_names[label] for label in classA]
# Figure 1: Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_Logistic.png')
plt.show

In [ ]:
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Bắt đầu đo thời gian
st = time.time()

# Step 5: Train an SVM Classifier  
classifier = SVC(kernel='rbf')  # Tạo đối tượng SVM, kernel có thể là 'linear', 'poly', 'rbf', hoặc 'sigmoid'
classifier.fit(features_train, y_train)  # Huấn luyện mô hình

# Step 6: Make Predictions  
y_pred = classifier.predict(features_test)  # Dự đoán với dữ liệu kiểm tra

# Step 7: Evaluate the Model  
print("Accuracy:", accuracy_score(y_test, y_pred))  # Đánh giá độ chính xác
print(classification_report(y_test, y_pred))  # In báo cáo phân loại với các chỉ số chính

# Kết thúc đo thời gian
nd = time.time()
print("Thời gian thực thi:", nd - st)  # In ra thời gian thực thi


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Step 7: Evaluate the Model and Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)  # Define conf_matrix here
class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
classA =[0,1,2,3,4]
# Map integer labels to class names
categories = [class_names[label] for label in classA]
# Figure 1: Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_SVM.png')
plt.show

**Trich xuat dac trung su dung inception resnetv2** 

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import warnings
import timm  # For Inception-ResNet-v2
import time
import cv2
from concurrent.futures import ThreadPoolExecutor

# Start timing
st = time.time()

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, label_map):
        self.image_paths = image_paths
        self.labels = labels
        self.label_map = label_map
        self.preprocess = transforms.Compose([
            transforms.Resize((299, 299)),  # Inception-ResNet-v2 requires 299x299
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            image_tensor = self.preprocess(image)
            label_idx = self.label_map[label]
            return image_tensor, label_idx, img_path
        except (UnidentifiedImageError, Exception) as e:
            print(f"Error loading image {img_path}: {e}")
            return torch.zeros(3, 299, 299), -1, img_path  # Dummy for invalid image

def extract_features(
    image_paths,
    labels,
    model_name='inception_resnet_v2',
    device=None,
    batch_size=16,
    num_workers=4,
    save_features=False,
    output_dir='features',
    label_map=None):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    num_workers = min(num_workers, os.cpu_count() or 1)

    if len(image_paths) != len(labels):
        raise ValueError("Number of image paths and labels must match.")
    if len(image_paths) == 0:
        print("No images provided.")
        return np.array([]), np.array([]), []

    try:
        model = timm.create_model(model_name, pretrained=True)
    except Exception as e:
        raise ValueError(f"Failed to load model {model_name}: {e}")

    model.eval()
    model.to(device)

    dataset = ImageDataset(image_paths, labels, label_map=label_map)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=(device != 'cpu'))

    features = []
    extracted_labels = []
    valid_paths = []

    for batch_tensors, batch_labels, batch_paths in tqdm(dataloader, desc="Extracting features"):
        batch_tensors = batch_tensors.to(device)
        valid_mask = batch_labels != -1
        if valid_mask.sum() == 0:
            continue

        with torch.no_grad():
            batch_features = model.forward_features(batch_tensors[valid_mask])
            batch_features = torch.nn.functional.adaptive_avg_pool2d(batch_features, (1, 1))
            batch_features = batch_features.view(batch_features.size(0), -1).cpu().numpy()

        features.append(batch_features)
        extracted_labels.extend(batch_labels[valid_mask].cpu().numpy())
        valid_paths.extend([batch_paths[i] for i in range(len(batch_paths)) if valid_mask[i]])

    if not features:
        print("No valid images processed.")
        return np.empty((0, model.num_features)), np.array([]), []

    features = np.concatenate(features, axis=0)
    extracted_labels = np.array(extracted_labels)

    if save_features:
        os.makedirs(output_dir, exist_ok=True)
        np.save(os.path.join(output_dir, 'features.npy'), features)
        np.save(os.path.join(output_dir, 'labels.npy'), extracted_labels)
        with open(os.path.join(output_dir, 'valid_paths.txt'), 'w') as f:
            f.write('\n'.join(valid_paths))
        print(f"Features saved to {output_dir}")

    print(f"Extracted {features.shape[0]} features.")
    return features, extracted_labels, valid_paths

    features = np.concatenate(features, axis=0)
    extracted_labels = np.array(extracted_labels)
def get_image_paths_and_labels(folder_path):
    image_paths = []
    labels = []
    label_map = {}
    for idx, label_name in enumerate(sorted(os.listdir(folder_path))):
        label_folder = os.path.join(folder_path, label_name)
        if os.path.isdir(label_folder):
            label_map[label_name] = idx
            for filename in os.listdir(label_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(label_folder, filename))
                    labels.append(label_name)
    return image_paths, labels, label_map

# Folder paths
train_folder = "/kaggle/input/lc25000/lung_colon_image_set/Train and Validation Set"
test_folder = "/kaggle/input/lc25000/lung_colon_image_set/Test Set"

# Load image paths and labels
x_train_paths, y_train, train_label_map = get_image_paths_and_labels(train_folder)
x_test_paths, y_test, test_label_map = get_image_paths_and_labels(test_folder)

# Confirm device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


# Extract test features
print("Extracting features for test set...")
xtest_features, y_test_labels, _ = extract_features(
    x_test_paths,
    y_test,
    model_name='inception_resnet_v2',
    device=device,
    batch_size=16,
    num_workers=4,
    save_features=True,
    output_dir='test_features',
    label_map=test_label_map
)
# Extract train features
print("Extracting features for training set...")
xtrain_features, y_train_labels, _ = extract_features(
    x_train_paths,
    y_train,
    model_name='inception_resnet_v2',
    device=device,
    batch_size=16,
    num_workers=4,
    save_features=True,
    output_dir='train_features',
    label_map=train_label_map
)

# Output shapes
print(f"Train features shape: {xtrain_features.shape}")
print(f"Train labels shape: {y_train_labels.shape}")
print(f"Test features shape: {xtest_features.shape}")
print(f"Test labels shape: {y_test_labels.shape}")

# End timing
end = time.time()
print(f"Total execution time: {end - st:.2f} seconds")


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KernelDensity
import numpy as np
import time
st=time.time()
class KDE_NaiveBayes:
    def __init__(self, bandwidth=0.05, default_class=None):
        self.bandwidth = bandwidth
        self.kde_models = {}
        self.classes = None
        self.priors = None
        self.default_class = default_class  # Fallback class if no valid prediction

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = {}
        n_samples = len(y)

        # Calculate prior probabilities
        for c in self.classes:
            self.priors[c] = np.sum(y == c) / n_samples
        
        # Fit KDE for each feature and each class, skip if no samples
        for c in self.classes:
            X_c = X[y == c]
            if X_c.shape[0] == 0:
                print(f"Warning: No samples found for class {c}. Skipping KDE fitting.")
                self.kde_models[c] = None
                continue
            self.kde_models[c] = []
            for feature_idx in range(X.shape[1]):
                kde = KernelDensity(bandwidth=self.bandwidth, kernel='gaussian')
                kde.fit(X_c[:, feature_idx].reshape(-1, 1))
                self.kde_models[c].append(kde)

    def predict(self, X):
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, len(self.classes)))

        for idx, c in enumerate(self.classes):
            log_prior = np.log(self.priors[c]) if self.priors[c] > 0 else -np.inf
            if self.kde_models[c] is None:
                log_probs[:, idx] = -np.inf
                continue
            log_likelihood = np.zeros(n_samples)
            for feature_idx, kde in enumerate(self.kde_models[c]):
                feature_log_prob = kde.score_samples(X[:, feature_idx].reshape(-1, 1))
                log_likelihood += feature_log_prob
            log_probs[:, idx] = log_prior+log_likelihood
        
        # Check if all probabilities are -inf
        if np.all(log_probs == -np.inf, axis=1).any():
            if self.default_class is not None:
                print(f"Warning: Some samples have no valid class probabilities. Using default class: {self.default_class}")
                y_pred = np.array([self.default_class if np.all(log_probs[i] == -np.inf) else self.classes[np.argmax(log_probs[i])] for i in range(n_samples)])
            else:
                raise ValueError("No valid predictions possible for some samples; all classes have zero probability. Set a default_class to handle this.")
        else:
            y_pred = self.classes[np.argmax(log_probs, axis=1)]
        
        return y_pred

# Example usage
classifier = KDE_NaiveBayes(bandwidth=0.05, default_class='Normal')  # Set a default class
classifier.fit(xtrain_features, y_train_labels)
y_pred = classifier.predict(xtest_features)

print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))
nd=time.time()
print("Thoi gian",nd-st)


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder  # <-- add this
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import time
st=time.time()
# Step 5: Train a Naive Bayes Classifier  
classifier = GaussianNB()  
classifier.fit(xtrain_features, y_train_labels)  # Training the classifier  

# Step 6: Make Predictions  
y_pred = classifier.predict(xtest_features)  

# Step 7: Evaluate the Model  
print("Accuracy:", accuracy_score(y_test_labels, y_pred))  
print(classification_report(y_test_labels, y_pred))  
nd=time.time()
print("Thoi gian thu thi", nd-st)

**2. Inception v3 Features**

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import warnings
import timm  # For Inception-ResNet-v2
import time
import cv2
from concurrent.futures import ThreadPoolExecutor

# Start timing
st = time.time()

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Custom Dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, label_map):
        self.image_paths = image_paths
        self.labels = labels
        self.label_map = label_map
        self.preprocess = transforms.Compose([
            transforms.Resize((299, 299)),  # Inception-ResNet-v2 requires 299x299
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            image_tensor = self.preprocess(image)
            label_idx = self.label_map[label]
            return image_tensor, label_idx, img_path
        except (UnidentifiedImageError, Exception) as e:
            print(f"Error loading image {img_path}: {e}")
            return torch.zeros(3, 299, 299), -1, img_path  # Dummy for invalid image

def extract_features(
    image_paths,
    labels,
    model_name='inception_resnet_v2',
    device=None,
    batch_size=16,
    num_workers=4,
    save_features=False,
    output_dir='features',
    label_map=None):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    num_workers = min(num_workers, os.cpu_count() or 1)

    if len(image_paths) != len(labels):
        raise ValueError("Number of image paths and labels must match.")
    if len(image_paths) == 0:
        print("No images provided.")
        return np.array([]), np.array([]), []

    try:
        model = timm.create_model(model_name, pretrained=True)
    except Exception as e:
        raise ValueError(f"Failed to load model {model_name}: {e}")

    model.eval()
    model.to(device)

    dataset = ImageDataset(image_paths, labels, label_map=label_map)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=(device != 'cpu'))

    features = []
    extracted_labels = []
    valid_paths = []

    for batch_tensors, batch_labels, batch_paths in tqdm(dataloader, desc="Extracting features"):
        batch_tensors = batch_tensors.to(device)
        valid_mask = batch_labels != -1
        if valid_mask.sum() == 0:
            continue

        with torch.no_grad():
            batch_features = model.forward_features(batch_tensors[valid_mask])
            batch_features = torch.nn.functional.adaptive_avg_pool2d(batch_features, (1, 1))
            batch_features = batch_features.view(batch_features.size(0), -1).cpu().numpy()

        features.append(batch_features)
        extracted_labels.extend(batch_labels[valid_mask].cpu().numpy())
        valid_paths.extend([batch_paths[i] for i in range(len(batch_paths)) if valid_mask[i]])

    if not features:
        print("No valid images processed.")
        return np.empty((0, model.num_features)), np.array([]), []

    features = np.concatenate(features, axis=0)
    extracted_labels = np.array(extracted_labels)

    if save_features:
        os.makedirs(output_dir, exist_ok=True)
        np.save(os.path.join(output_dir, 'features.npy'), features)
        np.save(os.path.join(output_dir, 'labels.npy'), extracted_labels)
        with open(os.path.join(output_dir, 'valid_paths.txt'), 'w') as f:
            f.write('\n'.join(valid_paths))
        print(f"Features saved to {output_dir}")

    print(f"Extracted {features.shape[0]} features.")
    return features, extracted_labels, valid_paths

    features = np.concatenate(features, axis=0)
    extracted_labels = np.array(extracted_labels)
def get_image_paths_and_labels(folder_path):
    image_paths = []
    labels = []
    label_map = {}
    for idx, label_name in enumerate(sorted(os.listdir(folder_path))):
        label_folder = os.path.join(folder_path, label_name)
        if os.path.isdir(label_folder):
            label_map[label_name] = idx
            for filename in os.listdir(label_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(label_folder, filename))
                    labels.append(label_name)
    return image_paths, labels, label_map

# Folder paths
train_folder = "/kaggle/input/lc25000/lung_colon_image_set/Train and Validation Set"
test_folder = "/kaggle/input/lc25000/lung_colon_image_set/Test Set"

# Load image paths and labels
x_train_paths, y_train, train_label_map = get_image_paths_and_labels(train_folder)
x_test_paths, y_test, test_label_map = get_image_paths_and_labels(test_folder)

# Confirm device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


# Extract test features
print("Extracting features for test set...")
xtest_features, y_test_labels, _ = extract_features(
    x_test_paths,
    y_test,
    model_name='inception_v3',
    device=device,
    batch_size=16,
    num_workers=4,
    save_features=True,
    output_dir='test_features',
    label_map=test_label_map
)
# Extract train features
print("Extracting features for training set...")
xtrain_features, y_train_labels, _ = extract_features(
    x_train_paths,
    y_train,
    model_name='inception_v3',
    device=device,
    batch_size=16,
    num_workers=4,
    save_features=True,
    output_dir='train_features',
    label_map=train_label_map
)

# Output shapes
print(f"Train features shape: {xtrain_features.shape}")
print(f"Train labels shape: {y_train_labels.shape}")
print(f"Test features shape: {xtest_features.shape}")
print(f"Test labels shape: {y_test_labels.shape}")

# End timing
end = time.time()
print(f"Total execution time: {end - st:.2f} seconds")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import time
st=time.time()
# Step 5: Train a Naive Bayes Classifier  
classifier = GaussianNB()  
classifier.fit(xtrain_features, y_train_labels)  # Training the classifier  

# Step 6: Make Predictions  
y_pred = classifier.predict(xtest_features)  

# Step 7: Evaluate the Model  
print("Accuracy:", accuracy_score(y_test_labels, y_pred))  
print(classification_report(y_test_labels, y_pred))  
nd=time.time()
print("Thoi gian thu thi", nd-st)

In [ ]:
from cuml.neighbors import KernelDensity as cuKDE  # GPU version
import cupy as cp  # GPU array (like numpy)
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import time

class KDE_NaiveBayes_GPU:
    def __init__(self, bandwidth=0.05, default_class=None):
        self.bandwidth = bandwidth
        self.kde_models = {}
        self.classes = None
        self.priors = None
        self.default_class = default_class

    def fit(self, X, y):
        X = cp.asarray(X)  # move to GPU
        y = cp.asarray(y)
        self.classes = cp.unique(y)
        self.priors = {}
        n_samples = len(y)

        for c in self.classes:
            class_mask = (y == c)
            self.priors[c.item()] = cp.sum(class_mask).item() / n_samples
            X_c = X[class_mask]
            if X_c.shape[0] == 0:
                print(f"Warning: No samples for class {c}. Skipping KDE.")
                self.kde_models[c.item()] = None
                continue
            self.kde_models[c.item()] = []
            for feature_idx in range(X.shape[1]):
                kde = cuKDE(kernel='gaussian', bandwidth=self.bandwidth)
                kde.fit(X_c[:, feature_idx])
                self.kde_models[c.item()].append(kde)

    def predict(self, X):
        X = cp.asarray(X)
        n_samples = X.shape[0]
        log_probs = cp.zeros((n_samples, len(self.classes)))

        for idx, c in enumerate(self.classes):
            class_value = c.item()
            log_prior = cp.log(self.priors[class_value]) if self.priors[class_value] > 0 else -cp.inf
            if self.kde_models[class_value] is None:
                log_probs[:, idx] = -cp.inf
                continue
            log_likelihood = cp.zeros(n_samples)
            for feature_idx, kde in enumerate(self.kde_models[class_value]):
                feature_log_prob = kde.score_samples(X[:, feature_idx])
                log_likelihood += feature_log_prob
            log_probs[:, idx] = log_prior + log_likelihood

        y_pred = []
        for i in range(n_samples):
            if cp.all(log_probs[i] == -cp.inf):
                if self.default_class is not None:
                    print(f"Warning: Sample {i} has no valid probabilities. Using default class.")
                    y_pred.append(self.default_class)
                else:
                    raise ValueError(f"Sample {i}: No valid prediction and no default_class provided.")
            else:
                pred_class = self.classes[cp.argmax(log_probs[i])].item()
                y_pred.append(pred_class)

        return np.array(y_pred)  # back to CPU

# ======================
# Example usage
# ======================
# Replace the following with your actual data:
# xtrain_features, xtest_features = shape (n_samples, n_features)
# y_train_labels, y_test_labels = shape (n_samples,)

# You must have your xtrain_features, y_train_labels, xtest_features, y_test_labels defined
# Example placeholders (remove or replace with real data):
# xtrain_features = np.random.rand(100, 10)
# y_train_labels = np.random.choice(['Normal', 'Anomaly'], 100)
# xtest_features = np.random.rand(20, 10)
# y_test_labels = np.random.choice(['Normal', 'Anomaly'], 20)

st = time.time()
classifier = KDE_NaiveBayes_GPU(bandwidth=0.05, default_class='Normal')
classifier.fit(xtrain_features, y_train_labels)
y_pred = classifier.predict(xtest_features)

print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))
print("Thời gian:", time.time() - st)


**VGG16**

In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
import warnings
import time
import cv2
from concurrent.futures import ThreadPoolExecutor

# Start timing
st = time.time()
warnings.filterwarnings("ignore")

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, label_map):
        self.image_paths = image_paths
        self.labels = labels
        self.label_map = label_map
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),  # VGG16 uses 224x224
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            image_tensor = self.preprocess(image)
            label_idx = self.label_map[label]
            return image_tensor, label_idx, img_path
        except (UnidentifiedImageError, Exception) as e:
            print(f"Error loading image {img_path}: {e}")
            return torch.zeros(3, 224, 224), -1, img_path  # Dummy for invalid image

def extract_features(image_paths, labels, device=None,
                     batch_size=16, num_workers=4, save_features=False,
                     output_dir='features', label_map=None):

    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    if len(image_paths) != len(labels):
        raise ValueError("Number of image paths and labels must match.")
    if len(image_paths) == 0:
        print("No images provided.")
        return np.array([]), np.array([]), []

    # Load pretrained VGG16
    model = models.vgg16(pretrained=True)
    model = model.features  # Use only the convolutional base
    model.eval().to(device)

    dataset = ImageDataset(image_paths, labels, label_map=label_map)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            num_workers=num_workers, shuffle=False,
                            pin_memory=(device != 'cpu'))

    features, extracted_labels, valid_paths = [], [], []

    for batch_tensors, batch_labels, batch_paths in tqdm(dataloader, desc="Extracting features"):
        batch_tensors = batch_tensors.to(device)
        valid_mask = batch_labels != -1
        if valid_mask.sum() == 0:
            continue

        with torch.no_grad():
            # Pass through VGG16 convolutional layers
            conv_feats = model(batch_tensors[valid_mask])
            # Apply global average pooling to get [B, 512]
            batch_feat = torch.nn.functional.adaptive_avg_pool2d(conv_feats, (1, 1)).view(conv_feats.size(0), -1)
            batch_feat = batch_feat.cpu().numpy()

        features.append(batch_feat)
        extracted_labels.extend(batch_labels[valid_mask].cpu().numpy())
        valid_paths.extend([batch_paths[i] for i in range(len(batch_paths)) if valid_mask[i]])

    if not features:
        print("No valid images processed.")
        return np.empty((0, 512)), np.array([]), []

    features = np.concatenate(features, axis=0)
    extracted_labels = np.array(extracted_labels)

    if save_features:
        os.makedirs(output_dir, exist_ok=True)
        np.save(os.path.join(output_dir, 'features.npy'), features)
        np.save(os.path.join(output_dir, 'labels.npy'), extracted_labels)
        with open(os.path.join(output_dir, 'valid_paths.txt'), 'w') as f:
            f.write('\n'.join(valid_paths))
        print(f"Features saved to {output_dir}")

    print(f"Extracted {features.shape[0]} features.")
    return features, extracted_labels, valid_paths

def get_image_paths_and_labels(folder_path):
    image_paths, labels, label_map = [], [], {}
    for idx, label_name in enumerate(sorted(os.listdir(folder_path))):
        label_folder = os.path.join(folder_path, label_name)
        if os.path.isdir(label_folder):
            label_map[label_name] = idx
            for filename in os.listdir(label_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(label_folder, filename))
                    labels.append(label_name)
    return image_paths, labels, label_map

# Folder paths
train_folder = "/kaggle/input/lc25000/lung_colon_image_set/Train and Validation Set"
test_folder = "/kaggle/input/lc25000/lung_colon_image_set/Test Set"

# Load image paths and labels
x_train_paths, y_train, train_label_map = get_image_paths_and_labels(train_folder)
x_test_paths, y_test, test_label_map = get_image_paths_and_labels(test_folder)

# Confirm device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Extract test features
print("Extracting features for test set...")
xtest_features, y_test_labels, _ = extract_features(
    x_test_paths, y_test, device=device, batch_size=16,
    num_workers=4, save_features=True,
    output_dir='test_features', label_map=test_label_map
)

# Extract train features
print("Extracting features for training set...")
xtrain_features, y_train_labels, _ = extract_features(
    x_train_paths, y_train, device=device, batch_size=16,
    num_workers=4, save_features=True,
    output_dir='train_features', label_map=train_label_map
)

# Output shapes
print(f"Train features shape: {xtrain_features.shape}")
print(f"Train labels shape: {y_train_labels.shape}")
print(f"Test features shape: {xtest_features.shape}")
print(f"Test labels shape: {y_test_labels.shape}")

# End timing
end = time.time()
print(f"Total execution time: {end - st:.2f} seconds")


Using device: cuda
Extracting features for test set...


Extracting features: 100%|██████████| 157/157 [00:15<00:00, 10.08it/s]


Features saved to test_features
Extracted 2499 features.
Extracting features for training set...


Extracting features: 100%|██████████| 1407/1407 [02:11<00:00, 10.74it/s]


Features saved to train_features
Extracted 22501 features.
Train features shape: (22501, 512)
Train labels shape: (22501,)
Test features shape: (2499, 512)
Test labels shape: (2499,)
Total execution time: 150.21 seconds


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import time
st=time.time()
# Step 5: Train a Naive Bayes Classifier  
classifier = GaussianNB()  
classifier.fit(xtrain_features, y_train_labels)  # Training the classifier  

# Step 6: Make Predictions  
y_pred = classifier.predict(xtest_features)  

# Step 7: Evaluate the Model  
print("Accuracy:", accuracy_score(y_test_labels, y_pred))  
print(classification_report(y_test_labels, y_pred))  
nd=time.time()
print("Thoi gian thu thi", nd-st)

Accuracy: 0.8803521408563425
              precision    recall  f1-score   support

           0       0.87      0.92      0.89       500
           1       0.97      0.91      0.94       500
           2       0.83      0.71      0.76       500
           3       0.98      0.99      0.98       500
           4       0.76      0.88      0.82       499

    accuracy                           0.88      2499
   macro avg       0.88      0.88      0.88      2499
weighted avg       0.88      0.88      0.88      2499

Thoi gian thu thi 0.1259174346923828


In [6]:
from cuml.neighbors import KernelDensity as cuKDE  # GPU version
import cupy as cp  # GPU array (like numpy)
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import time

class KDE_NaiveBayes_GPU:
    def __init__(self, bandwidth=0.05, default_class=None):
        self.bandwidth = bandwidth
        self.kde_models = {}
        self.classes = None
        self.priors = None
        self.default_class = default_class

    def fit(self, X, y):
        X = cp.asarray(X)  # move to GPU
        y = cp.asarray(y)
        self.classes = cp.unique(y)
        self.priors = {}
        n_samples = len(y)

        for c in self.classes:
            class_mask = (y == c)
            self.priors[c.item()] = cp.sum(class_mask).item() / n_samples
            X_c = X[class_mask]
            if X_c.shape[0] == 0:
                print(f"Warning: No samples for class {c}. Skipping KDE.")
                self.kde_models[c.item()] = None
                continue
            self.kde_models[c.item()] = []
            for feature_idx in range(X.shape[1]):
                kde = cuKDE(kernel='gaussian', bandwidth=self.bandwidth)
                kde.fit(X_c[:, feature_idx])
                self.kde_models[c.item()].append(kde)

    def predict(self, X):
        X = cp.asarray(X)
        n_samples = X.shape[0]
        log_probs = cp.zeros((n_samples, len(self.classes)))

        for idx, c in enumerate(self.classes):
            class_value = c.item()
            log_prior = cp.log(self.priors[class_value]) if self.priors[class_value] > 0 else -cp.inf
            if self.kde_models[class_value] is None:
                log_probs[:, idx] = -cp.inf
                continue
            log_likelihood = cp.zeros(n_samples)
            for feature_idx, kde in enumerate(self.kde_models[class_value]):
                feature_log_prob = kde.score_samples(X[:, feature_idx])
                log_likelihood += feature_log_prob
            log_probs[:, idx] = log_prior + log_likelihood

        y_pred = []
        for i in range(n_samples):
            if cp.all(log_probs[i] == -cp.inf):
                if self.default_class is not None:
                    print(f"Warning: Sample {i} has no valid probabilities. Using default class.")
                    y_pred.append(self.default_class)
                else:
                    raise ValueError(f"Sample {i}: No valid prediction and no default_class provided.")
            else:
                pred_class = self.classes[cp.argmax(log_probs[i])].item()
                y_pred.append(pred_class)

        return np.array(y_pred)  # back to CPU

# ======================
# Example usage
# ======================
# Replace the following with your actual data:
# xtrain_features, xtest_features = shape (n_samples, n_features)
# y_train_labels, y_test_labels = shape (n_samples,)

# You must have your xtrain_features, y_train_labels, xtest_features, y_test_labels defined
# Example placeholders (remove or replace with real data):
# xtrain_features = np.random.rand(100, 10)
# y_train_labels = np.random.choice(['Normal', 'Anomaly'], 100)
# xtest_features = np.random.rand(20, 10)
# y_test_labels = np.random.choice(['Normal', 'Anomaly'], 20)

st = time.time()
classifier = KDE_NaiveBayes_GPU(bandwidth=0.05, default_class='Normal')
classifier.fit(xtrain_features, y_train_labels)
y_pred = classifier.predict(xtest_features)

print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print(classification_report(y_test_labels, y_pred))
print("Thời gian:", time.time() - st)


Accuracy: 0.923969587835134
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       500
           1       0.98      0.92      0.95       500
           2       0.87      0.87      0.87       500
           3       0.98      0.99      0.99       500
           4       0.85      0.91      0.88       499

    accuracy                           0.92      2499
   macro avg       0.93      0.92      0.92      2499
weighted avg       0.93      0.92      0.92      2499

Thời gian: 30.607917308807373
